In [1]:
from typing import List, Optional, Generator
import typing
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq
import operator

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

CF Div 2 - 15th Aug 2024

In [4]:
def closest_point(N: int, points: list[int]) -> bool:
    return N == 2 and abs(points[0] - points[1]) > 1

# Testing the solution
assert closest_point(2, [3,8]) == True
assert closest_point(2, [5,6]) == False
assert closest_point(6, [1,2,3,4,5,10]) == False

Upsolvng CF Contest - Div 2

In [5]:
def game_with_doors(AL: int, AR: int, BL: int, BR: int) -> int:
    """
    For the overlapping portion, all doors must be closed
    For non overlapping portion, suffices to close just one door at each side
    """
    # Check for overlap
    oL, oR = max(AL, BL), min(AR, BR)
    if oL <= oR:
        locks = oR - oL
        if abs(AL - BL) > 0:
            locks += 1
        if abs(AR  - BR) > 0:
            locks += 1
        return locks
    else:
        return 1

# Testing the solution
assert game_with_doors(1, 2, 3, 4) == 1
assert game_with_doors(2, 5, 2, 5) == 3
assert game_with_doors(3, 7, 6, 7) == 2
assert game_with_doors(4, 5, 2, 8) == 3

In [6]:
def splitting_items(N: int, K: int, costs: list[int]) -> int:
    """
    * Playing optimally implies that
        - Alice tries to pick up the largest costing item available
        - Bob picks up the largest costing item available
        - Alice picks up items at odd indices, Bob at even indices (post sorting)
    * To minimize `A - B`, Bob should increase cost of items at even indices
    * We can update delta on the fly until K > 0
    """
    costs.sort()
    result = 0
    while costs:
        if len(costs) >= 2:
            delta = costs.pop() - costs.pop()
            if K < delta:
                result += delta - K
            K = max(0, K - delta)
        else:
            result += costs.pop()

    return result

# Testing the solution
assert splitting_items(3, 0, [10, 15, 12]) == 13
assert splitting_items(2, 5, [1, 10]) == 4
assert splitting_items(4, 6, [3, 1, 2, 4]) == 0
assert splitting_items(2, 4, [6, 9]) == 0
assert splitting_items(3, 1, [1, 1, 1]) == 1

In [7]:
def colored_portals(N: int, Q: int, cities: list[str], queries: list[tuple[int, int]]) -> list[int]:
    """
    If cities of i, j share a common color, shortest path to reach would abs(i - j)
    Otherwise, we check for common colors between target:
        - If no common colors return -1
        - Elif if there exists a common point between i, j return abs(i - j)
        - Else search for points to left of i and right i with common portal and check both distances, return least of two (use binary search)
    """

    # Store to hashmap for quick lookup times
    portal_groups: collections.defaultdict[str, list] = collections.defaultdict(list)
    for i, portal_type in enumerate(cities):
        portal_groups[portal_type].append(i)

    # Process the queries
    results: list[int] = []
    for i, j in queries:
        i, j = i - 1, j - 1
        a, b, c, d = cities[i][0], cities[i][1], cities[j][0], cities[j][1]

        # All 4 distinct portals
        if len(set([a, b, c, d])) == 4:
            distance = math.inf
            for comb in [(a, c), (a, d), (b, c), (b, d)]:
                common = "".join(sorted(comb))
                idx = bisect.bisect(portal_groups[common], i)
                left, right = idx - 1, idx
                if 0 <= left < len(portal_groups[common]):
                    distance = min(distance, abs(portal_groups[common][left] - i) + abs(portal_groups[common][left] - j))

                if 0 <= right < len(portal_groups[common]):
                    distance = min(distance, abs(portal_groups[common][right] - i) + abs(portal_groups[common][right] - j))

            results.append(int(distance) if not math.isinf(distance) else -1)

        # Atleast one color in common
        else:
           results.append(abs(j - i))

    return results

# Testing the solution
assert colored_portals(6, 6, ["RY", "RY", "GY", "RY", "GB", "RY"], [(2,5), (4,5), (1,4), (4,1), (5,4), (5,2)]) == [3,3,3,3,3,3]
assert colored_portals(9, 3, ["RY", "RY", "BR", "BR", "BR", "BR", "GY", "GY", "RY"], [(3,7), (2,5), (6,7)]) == [6,3,5]
assert colored_portals(4, 5, ["BR","BR","GY","GR"], [(1,2), (3,1), (4,4), (1,4), (4,2)]) == [1,4,0,3,2]

Find MEX of a set

In [8]:
def MEX(N: int, nums: list[int]) -> int:
    """
    There is a more optimized version for cases where nums are frequently updated.
    """
    xor = functools.reduce(operator.xor, nums, 0)
    missing = functools.reduce(operator.xor, range(N), xor)
    return missing

# Testing the solution
assert MEX(5, [0,2,3,4]) == 1
assert MEX(3, [1,2]) == 0
assert MEX(3, [2,0]) == 1
assert MEX(4, [2,0,1]) == 3

In [9]:
# Leetcode Weekly: 18th Aug 2024

In [10]:
def countKConstraintSubstrings(S: str, K: int) -> int:
    N = len(S)
    total = 0
    for i in range(N):
        zeros = ones = 0
        for j in range(i, N):
            if S[j] == '1':
                ones += 1
            else:
                zeros += 1
            if ones <= K or zeros <= K:
                total += 1
            else:
                break
    return total

# Testing the solution
assert countKConstraintSubstrings("10101", 1) == 12
assert countKConstraintSubstrings("1010101", 2) == 25
assert countKConstraintSubstrings("11111", 1) == 15

In [11]:
def maxEnergyBoost(energyDrinkA: list[int], energyDrinkB: list[int]) -> int:
    @functools.cache
    def backtrack(i: int, prev: int = 0) -> int:
        if i == N:
            return 0
        else:
            max_energy = backtrack(i + 1, 0)
            if prev != 2:
                max_energy = max(max_energy, energyDrinkA[i] + backtrack(i + 1, 1))
            if prev != 1:
                max_energy = max(max_energy, energyDrinkB[i] + backtrack(i + 1, 2))
            return max_energy

    N = len(energyDrinkA)
    return backtrack(0)

# Testing the solution
assert maxEnergyBoost([1,3,1], [3,1,1]) == 5
assert maxEnergyBoost([4,1,1], [1,1,3]) == 7

CP Practice - 19th Aug 2024

In [12]:
def learning_languages(N: int, M: int, languages_known: list[tuple[int, list[int]]]) -> int:
    # Store the speakers who speak a particular language
    language_speakers: collections.defaultdict[int, list[int]] = collections.defaultdict(list)
    for employee, (count, languages) in enumerate(languages_known):
        for language in languages:
            language_speakers[language].append(employee)

    # Perform a DFS traversal, we need to count the number of distinct groups
    visited: set[int] = set()
    know_nothings, groups = 0, 0
    for i in range(N):
        if i not in visited:
            if languages_known[i][0]:
                groups += 1
                stack: list[int] = [i]
                visited.add(i)
                while stack:
                    emp = stack.pop()
                    for language in languages_known[emp][1]:
                        for next_emp in language_speakers[language]:
                            if next_emp not in visited:
                                stack.append(next_emp)
                                visited.add(next_emp)
            else:
                know_nothings += 1

    return max(groups - 1, 0) + know_nothings

# Testing the solution
assert learning_languages(8, 7, [(0, []), (3, [1,2,3]), (1, [1]), (2, [5,4]), (2, [6,7]), (1, [3]), (2, [7,4]), (1, [1])]) == 2
assert learning_languages(2, 2, [(1, [2]), (0, [])]) == 1
assert learning_languages(5, 5, [(1, [2]), (2, [2,3]), (2, [3,4]), (2, [4,5]), (1, [5])]) == 0
assert learning_languages(2, 2, [(0, []), (0, [])]) == 2
assert learning_languages(3, 2, [(0, []), (1, [1]), (1, [2])]) == 2

CP Practice - 20th Aug 2024

In [13]:
def countSeniors(details: list[str]) -> int:
    return sum(map(lambda x: int(x[11:13]) > 60, details))

# Testing the solution
assert countSeniors(["7868190130M7522","5303914400F9211","9273338290F4010"]) == 2
assert countSeniors(["1313579440F2036","2921522980M5644"]) == 0

In [14]:
def canBeEqual(target: list[int], arr: list[int]) -> bool:
    return collections.Counter(target) == collections.Counter(arr)

# Testing the solution
assert canBeEqual([1,2,3,4], [2,4,1,3]) == True
assert canBeEqual([7], [7]) == True
assert canBeEqual([3,7,9], [3,7,11]) == False

CP Practice: 21th Aug 2024

In [15]:
def bertown_subway(N: int, stations: list[int]) -> int:
    # Compute the number of disjoint stations
    groups: collections.defaultdict[int, set[int]] = collections.defaultdict(set)
    color: int = 0
    visited: set[int] = set()
    for i in range(N):
        if i not in visited:
            curr, color = i, color + 1
            while curr not in visited:
                visited.add(curr)
                groups[color].add(curr)
                curr = stations[curr] - 1

    # We can combine any two groups - to maximize, join the last two groups
    group_sizes: list[int] = sorted(map(len, groups.values()))
    return sum(map(lambda x: x ** 2, group_sizes[:-2])) + sum(group_sizes[-2:]) ** 2

# Testing the solution
assert bertown_subway(5, [1,5,4,3,2]) == 17
assert bertown_subway(3, [2,1,3]) == 9

In [16]:
def findComplement(num: int) -> int:
    result, power = 0, 1
    while num:
        result += power if not num & 1 else 0
        num, power = num >> 1, power << 1
    return result

# Testing the solution
assert findComplement(5) == 2
assert findComplement(1) == 0

CP Practice: 24th Aug 2024

In [17]:
def k_complete_word(N: int, K: int, word: str) -> int:
    updates = 0
    visited: set[int] = set()
    for i in range(N):
        if i not in visited:
            chars: list[int] = [0] * 26
            max_, count = 0, 0
            for j in range(i, N, K):
                if j not in visited:
                    visited.add(j)
                    chars[ord(word[j]) - ord('a')] += 1
                    max_, count = max(max_, chars[ord(word[j]) - ord('a')]), count + 1
                if N - j - 1 not in visited:
                    visited.add(N - j - 1)
                    chars[ord(word[N - j - 1]) - ord('a')] += 1
                    max_, count = max(max_, chars[ord(word[N - j - 1]) - ord('a')]), count + 1
            updates += (count - max_)

    return updates

# Testing the solution
assert k_complete_word(36, 9, "hippopotomonstrosesquippedaliophobia") == 23
assert k_complete_word(21, 7, "wudixiaoxingxingheclp") == 16
assert k_complete_word(6, 2, "abaaba") == 2
assert k_complete_word(6, 3, "abaaba") == 0